In [11]:
from rfpipe import pipeline
# Updated for rfpipe version 1.3.1
from rfpipe import candidates
import rfpipe
import numpy as np 
import pylab as plt
import matplotlib
import logging
import glob
logger = logging.getLogger('rfpipe')
%matplotlib inline
from refine_utils import cd_refine, max_timewindow

In [12]:
params = {
        'axes.labelsize' : 14,
        'font.size' : 9,
        'legend.fontsize': 12,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12,
        'text.usetex': False,
        'figure.figsize': [20, 15]
        }
matplotlib.rcParams.update(params)

In [13]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Set up data paths

In [14]:
datadir = '/lustre/aoc/projects/fasttransients/staging/'
sdmname = 'realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566269152540'
#'realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566270772899'
workdir = '/lustre/aoc/observers/nm-10222/rfgpu/Refinement/'
gainpath = '/home/mchammer/evladata/telcal/'

In [15]:
datasetId = '{0}'.format('_'.join(sdmname.split('_')[1:-1]))
# # set the paths to the gainfile
gainname = datasetId + '.GN'
logging.info('Searching for the gainfile {0} in {1}'.format(gainname, gainpath))
for path, dirs, files in os.walk(gainpath):
    for f in filter(lambda x: gainname in x, files):
        gainfile = os.path.join(path, gainname)
        break

2019-08-20 21:48:55,368 - root - INFO - Searching for the gainfile 19A-397.sb37006961.eb37092350.58715.036993946764.GN in /home/mchammer/evladata/telcal/


KeyboardInterrupt: 

In [16]:
gainfile = '/home/mchammer/evladata/telcal/2019/08/19A-397.sb37006961.eb37092350.58715.036993946764.GN'

# Inputs 

In [17]:
preffile = datadir+'realfast.yml' # Not being used currently
search_sigma = 7
classify = True
sdmscan = 1
dm_steps = 100
dm_frac = 0.2

In [19]:
# prefs={'gainfile': gainfile, 'saveplots': False, 'savenoise': False, 'savesols': False,
#        'savecandcollection': False, 'savecanddata': False,
#        'applyonlineflags': True, 'fftmode': 'cuda', 'clustercands': False, 'maxdm':2000, 'dtarr':[1,2,4,8]}

# Search on a larger timewindow 

In [20]:
try:
    st = rfpipe.state.State(sdmfile=os.path.join(datadir, sdmname), sdmscan=sdmscan, 
                            preffile = preffile, name='NRAOdefaultS')
except AssertionError:
    prefs['maxdm'] = 500
    st = rfpipe.state.State(sdmfile=os.path.join(datadir, sdmname), sdmscan=sdmscan, 
                            preffile = preffile, name='NRAOdefaultS')

st.prefs.sigma_image1 = search_sigma
st.prefs.workdir = workdir
st.prefs.gainfile = gainfile
st.prefs.saveplots = False

# Uncomment the following line to run search without RFI flagging. 
# st.prefs.flaglist = []

tw = max_timewindow(st)
if tw > st.prefs.timewindow:
    try:
        logging.info('Setting timewindow to {0}'.format(tw))
        st.prefs.timewindow = tw
    except AssertionError:
        pass

2019-08-20 21:49:12,519 - rfpipe.preferences - INFO - Parsing preffile for preference set NRAOdefaultS
2019-08-20 21:49:12,539 - rfpipe.metadata - INFO - Reading metadata from /lustre/aoc/projects/fasttransients/staging/realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566269152540, scan 1
2019-08-20 21:49:12,773 - rfpipe.state - INFO - Metadata summary:
2019-08-20 21:49:12,774 - rfpipe.state - INFO - 	 Working directory and fileroot: /lustre/aoc/observers/nm-10222/rfgpu/realfast_analysis/refinement_notebooks, realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566269152540.1.1
2019-08-20 21:49:12,774 - rfpipe.state - INFO - 	 Using scan 1, source SgrA E
2019-08-20 21:49:12,774 - rfpipe.state - INFO - 	 nants, nbl: 25, 300
2019-08-20 21:49:12,775 - rfpipe.state - INFO - 	 nchan, nspw: 896, 16
2019-08-20 21:49:12,777 - rfpipe.state - INFO - 	 Freq range: 1.996 -- 4.002
2019-08-20 21:49:12,777 - rfpipe.state - INFO - 	 Scan has 357 ints (3.6 s) and inttime 0.010 s
2019-08

In [21]:
cc = rfpipe.pipeline.pipeline_scan(st)

2019-08-20 21:49:17,178 - rfpipe.source - INFO - Reading segment 0 of scanId realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566269152540.1.1
2019-08-20 21:49:17,179 - rfpipe.source - INFO - Reading scan 1, segment 0/0, times 1566269152.5400002 to 1566269156.1100001
2019-08-20 21:49:17,180 - rfpipe.source - INFO - Reading 357 ints starting at int 0
2019-08-20 21:49:31,042 - rfpipe.source - INFO - Read data with zero fraction of 0.006 for segment 0
2019-08-20 21:49:33,245 - rfpipe.flagging - INFO - No flagged antennas in time range 58715.11519143519-58715.11523275463 
2019-08-20 21:49:34,345 - rfpipe.source - INFO - Applying calibration with /home/mchammer/evladata/telcal/2019/08/19A-397.sb37006961.eb37092350.58715.036993946764.GN
2019-08-20 21:49:34,466 - rfpipe.calibration - INFO - Read telcalfile /home/mchammer/evladata/telcal/2019/08/19A-397.sb37006961.eb37092350.58715.036993946764.GN with 3 sources, 59 times, 32 IFIDs, and 25 antennas
2019-08-20 21:49:34,472 - rfpipe.cal

2019-08-20 21:50:24,331 - rfpipe.search - INFO - Imaging 313 ints (0-312) in seg 0 at DM/dt 565.5/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:24,654 - rfpipe.search - INFO - Imaging 312 ints (0-311) in seg 0 at DM/dt 581.6/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:24,977 - rfpipe.search - INFO - Imaging 311 ints (0-310) in seg 0 at DM/dt 597.9/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:25,300 - rfpipe.search - INFO - Imaging 309 ints (0-308) in seg 0 at DM/dt 614.1/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:25,619 - rfpipe.search - INFO - Imaging 308 ints (0-307) in seg 0 at DM/dt 630.2/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:25,938 - rfpipe.search - INFO - Imaging 307 ints (0-306) in seg 0 at DM/dt 646.5/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:26,256 - rfpipe.search - INFO - Imaging 306 ints (0-305) in seg 0 at DM/dt 662.7/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21

2019-08-20 21:50:39,488 - rfpipe.search - INFO - Imaging 247 ints (0-246) in seg 0 at DM/dt 1407.9/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:39,612 - rfpipe.search - INFO - Got one! SNR1 7.1 candidate at (0, 120, 87, 0, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:50:39,746 - rfpipe.search - INFO - Imaging 246 ints (0-245) in seg 0 at DM/dt 1424.1/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:39,870 - rfpipe.search - INFO - Got one! SNR1 7.7 candidate at (0, 120, 88, 0, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:50:40,003 - rfpipe.search - INFO - Imaging 245 ints (0-244) in seg 0 at DM/dt 1440.3/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:40,258 - rfpipe.search - INFO - Imaging 244 ints (0-243) in seg 0 at DM/dt 1456.6/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:40,383 - rfpipe.search - INFO - Got one! SNR1 7.7 candidate at (0, 121, 90, 0, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:50:40,513 - rfpipe.

2019-08-20 21:50:44,552 - rfpipe.search - INFO - Got one! SNR1 12.0 candidate at (0, 120, 107, 0, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:50:44,554 - rfpipe.search - INFO - Got one! SNR1 7.9 candidate at (0, 121, 107, 0, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:50:44,662 - rfpipe.search - INFO - Imaging 221 ints (0-220) in seg 0 at DM/dt 1749.1/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:44,784 - rfpipe.search - INFO - Got one! SNR1 16.8 candidate at (0, 119, 108, 0, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:50:44,786 - rfpipe.search - INFO - Got one! SNR1 11.6 candidate at (0, 120, 108, 0, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:50:44,788 - rfpipe.search - INFO - Got one! SNR1 8.2 candidate at (0, 121, 108, 0, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:50:44,895 - rfpipe.search - INFO - Imaging 219 ints (0-218) in seg 0 at DM/dt 1765.3/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:45,018 - rfpipe.search - I

2019-08-20 21:50:48,024 - rfpipe.search - INFO - Imaging 202 ints (0-201) in seg 0 at DM/dt 1993.0/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:48,145 - rfpipe.search - INFO - Got one! SNR1 9.0 candidate at (0, 117, 123, 0, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:50:48,147 - rfpipe.search - INFO - Got one! SNR1 11.2 candidate at (0, 118, 123, 0, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:50:48,149 - rfpipe.search - INFO - Got one! SNR1 9.8 candidate at (0, 119, 123, 0, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:50:48,238 - rfpipe.search - INFO - Imaging 201 ints (0-200) in seg 0 at DM/dt 2000.0/1 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:50:48,359 - rfpipe.search - INFO - Got one! SNR1 9.1 candidate at (0, 117, 124, 0, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:50:48,361 - rfpipe.search - INFO - Got one! SNR1 10.9 candidate at (0, 118, 124, 0, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:50:48,363 - rfpipe.search - IN

...

2019-08-20 21:51:18,439 - rfpipe.search - INFO - Imaging 39 ints (0-38) in seg 0 at DM/dt 516.9/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:18,486 - rfpipe.search - INFO - Imaging 39 ints (0-38) in seg 0 at DM/dt 533.1/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:18,532 - rfpipe.search - INFO - Imaging 39 ints (0-38) in seg 0 at DM/dt 549.2/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:18,579 - rfpipe.search - INFO - Imaging 39 ints (0-38) in seg 0 at DM/dt 565.5/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:18,625 - rfpipe.search - INFO - Imaging 39 ints (0-38) in seg 0 at DM/dt 581.6/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:18,671 - rfpipe.search - INFO - Imaging 39 ints (0-38) in seg 0 at DM/dt 597.9/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:18,718 - rfpipe.search - INFO - Imaging 38 ints (0-37) in seg 0 at DM/dt 614.1/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:18,764 - r

2019-08-20 21:51:20,422 - rfpipe.search - INFO - Got one! SNR1 7.9 candidate at (0, 15, 77, 3, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:51:20,445 - rfpipe.search - INFO - Imaging 32 ints (0-31) in seg 0 at DM/dt 1262.1/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:20,463 - rfpipe.search - INFO - Got one! SNR1 8.2 candidate at (0, 15, 78, 3, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:51:20,486 - rfpipe.search - INFO - Imaging 32 ints (0-31) in seg 0 at DM/dt 1278.2/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:20,503 - rfpipe.search - INFO - Got one! SNR1 8.9 candidate at (0, 15, 79, 3, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:51:20,527 - rfpipe.search - INFO - Imaging 32 ints (0-31) in seg 0 at DM/dt 1294.5/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:20,544 - rfpipe.search - INFO - Got one! SNR1 8.8 candidate at (0, 15, 80, 3, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:51:20,567 - rfpipe.search - INFO - Ima

2019-08-20 21:51:21,594 - rfpipe.search - INFO - Imaging 27 ints (0-26) in seg 0 at DM/dt 1749.1/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:21,611 - rfpipe.search - INFO - Got one! SNR1 13.9 candidate at (0, 15, 108, 3, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:51:21,629 - rfpipe.search - INFO - Imaging 27 ints (0-26) in seg 0 at DM/dt 1765.3/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:21,646 - rfpipe.search - INFO - Got one! SNR1 13.9 candidate at (0, 15, 109, 3, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:51:21,664 - rfpipe.search - INFO - Imaging 27 ints (0-26) in seg 0 at DM/dt 1781.6/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:21,682 - rfpipe.search - INFO - Got one! SNR1 14.2 candidate at (0, 15, 110, 3, 0) and (l, m) = (-0.00006, -0.00001)
2019-08-20 21:51:21,699 - rfpipe.search - INFO - Imaging 27 ints (0-26) in seg 0 at DM/dt 1797.8/8 with image 2048x2048 (uvres 83) on GPU 0
2019-08-20 21:51:21,717 - rfpipe.searc

2019-08-20 21:51:30,826 - rfpipe.reproduce - INFO - Cluster 0/10 has 206 candidates and max detected SNR 19.2 at (0, 119, 110, 0, 0)
2019-08-20 21:51:30,976 - rfpipe.search - INFO - Correcting by delay/resampling 139/1 ints in parallel mode
2019-08-20 21:51:31,121 - rfpipe.search - INFO - Resampling data of length 357 by a factor of 1
2019-08-20 21:51:31,928 - rfpipe.search - INFO - Dedispersing up to delay shift of 139 integrations
2019-08-20 21:51:34,138 - rfpipe.reproduce - INFO - Calculated snrk of 1.4816033928874004 after detection. Adding it to CandData.
2019-08-20 21:51:34,203 - rfpipe.search - INFO - Calculating FFT wisdom...
2019-08-20 21:51:39,239 - rfpipe.candidates - INFO - Making 1 candidate plots.
2019-08-20 21:51:39,265 - rfpipe.candidates - INFO - Plotting candloc (0, 119, 110, 0, 0) with SNR 19.3 and image/data shapes: (2048, 2048)/(156, 896, 2)
2019-08-20 21:51:39,266 - rfpipe.candidates - INFO - Peak (RA, Dec): (17 45 40.16190520448788, -29 0 29.383335046337322)
2019

KeyboardInterrupt: 

In [ ]:
try:
    assert len(cc.locs) 
except AssertionError as err:
    logging.exception("No candidates found in the search.")
    raise err

In [ ]:
# Cluster and visualize Candidates
cc.prefs.clustercands = True
cc_clustered, clusterer = rfpipe.candidates.cluster_candidates(cc, returnclusterer=True)
rfpipe.candidates.visualize_clustering(cc_clustered, clusterer)

## Reproduce candidate and generate refinement plots

In [ ]:
clusters = cc_clustered.array['cluster'].astype(int)
cl_rank, cl_count = candidates.calc_cluster_rank(cc_clustered)
calcinds = np.unique(np.where(cl_rank == 1)[0]).tolist()
logging.info("Reproducing cands at {0} cluster peaks of SNR: {1}".format(len(calcinds),
                                                                         cc_clustered.snrtot[calcinds]))

In [ ]:
for ind in calcinds:
    candloc = cc_clustered.locs[ind]
    cd = rfpipe.reproduce.pipeline_canddata(st, candloc)
    logging.info(f'Processing candidate at candloc {cd.loc}')
    if cd.data.any():
        if classify:
            payload = candidates.cd_to_fetch(cd, classify=True, save_png=False, show=False, mode = 'GPU')
            logging.info('FETCH FRB Probability of the candidate {0} is {1}'.format(cd.candid, payload))
        logging.info('Generating Refinement plots')
        cd_refine(cd)
    else:
        logging.warning('Canddata is empty. Skipping Candidate')    

# Search with finer DM grid

In [ ]:
# DM of the candidate to be refined. 
dm = 100

In [ ]:
#set the "refined" preferences
curr_prefs = st.prefs
curr_prefs.dmarr = list(np.linspace(dm-dm_frac*dm, dm+dm_frac*dm, dm_steps))
st = rfpipe.state.State(sdmfile=os.path.join(datadir, sdmname), sdmscan=sdmscan, inprefs=curr_prefs)
cc = rfpipe.pipeline.pipeline_scan(st)

In [ ]:
try:
    assert len(cc.locs) 
except AssertionError as err:
    logging.exception("No candidates found in the search.")
    raise err

In [ ]:
# Visualise clustering
cc.prefs.clustercands = True
cc_clustered, clusterer = rfpipe.candidates.cluster_candidates(cc, returnclusterer=True)
rfpipe.candidates.visualize_clustering(cc_clustered, clusterer)

In [ ]:
clusters = cc_clustered.array['cluster'].astype(int)
cl_rank, cl_count = candidates.calc_cluster_rank(cc_clustered)
calcinds = np.unique(np.where(cl_rank == 1)[0]).tolist()
logging.info("Reproducing cands at {0} cluster peaks of SNR: {1}".format(len(calcinds),
                                                                         cc_clustered.snrtot[calcinds]))

# Reproduce and Generate refinement plots of fine-DM searched candidates

In [ ]:
for ind in calcinds:
    candloc = cc_clustered.locs[ind]
    cd = rfpipe.reproduce.pipeline_canddata(st, candloc)
    logging.info(f'Processing candidate at candloc {cd.loc}')
    if cd.data.any():
        if classify:
            payload = candidates.cd_to_fetch(cd, classify=True, save_png=False, show=False, mode = 'GPU')
            logging.info('FETCH FRB Probability of the candidate {0} is {1}'.format(cd.candid, payload))
        logging.info('Generating Refinement plots')
        cd_refine(cd)
    else:
        logging.warning('Canddata is empty. Skipping Candidate')    